In [1]:
!pip install wget 

In [2]:
!pip install scikit-learn 

In [3]:
!pip install "ibm-watson-machine-learning>=1.0.310" 

In [7]:
credentials = {
    "url":"https://eu-de.ml.cloud.ibm.com" ,
    "apikey":"edIGuoCn8w2m_Drszf3wVzy9m0_KkCWifHWbY7x27nyE"
}

In [8]:
import os

In [9]:
try:
    project_id=os.environ["PROJECT_ID"]

except KeyError:
    project_id = input("please enter your project_id:")

In [10]:
project_id

'662f4904-0344-4ea9-b975-da4745bbb2a1'

In [12]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='IcXY9sbCKgaK7ZpjkV7MdDtT9941lWVTf0ejXeyx44mc',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.eu-de.cloud-object-storage.appdomain.cloud')

bucket = 'lashassandbox-donotdelete-pr-7asd0ktkznumzv'
object_key = 'legal_sentiment_dataset.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_csv(body)
data.head(10)


In [16]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='IcXY9sbCKgaK7ZpjkV7MdDtT9941lWVTf0ejXeyx44mc',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.eu-de.cloud-object-storage.appdomain.cloud')

bucket = 'lashassandbox-donotdelete-pr-7asd0ktkznumzv'
object_key = 'legal_sentiment_dataset.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_csv(body)
data.head(10)


,ID,Phrase,Sentiment
0,1,The plaintiff's claims are dismissed.,-1
1,2,The contract is deemed valid and enforceable.,1
2,3,The appeal is denied due to lack of merit.,-1
3,4,The legal team submitted additional evidence.,0
4,5,The appeal is under consideration.,0
5,6,The settlement agreement is fair and reasonable.,1
6,7,The appeal is under consideration.,0
7,8,The settlement agreement is fair and reasonable.,1
8,9,The court reviewed the documentation.,0
9,10,The contract is deemed valid and enforceable.,1


In [18]:
label_map={
    -1:"negative",
    0:"neural",
    1:"positive"
}

In [20]:
data.value_counts(['Sentiment'])

Sentiment
 1           184
-1           167
 0           149
dtype: int64

In [33]:
from sklearn.model_selection import train_test_split

data_train,data_test,y_train,y_test=train_test_split(data['Phrase'],
                                                     data['Sentiment'],
                                                     test_size=0.3,
                                                     random_state=33,
                                                     stratify=data['Sentiment'])
print(type(data_train))
data_train=pd.DataFrame(data_train)
data_test=pd.DataFrame(data_test)


<class 'pandas.core.series.Series'>


In [25]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

print([model.name for model in ModelTypes])

['FLAN_T5_XXL', 'FLAN_UL2', 'MT0_XXL', 'GPT_NEOX', 'MPT_7B_INSTRUCT2']


In [26]:
model_id=ModelTypes.FLAN_T5_XXL

In [27]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

In [28]:
parameters={
    GenParams.DECODING_METHOD:"Greedy",
    GenParams.RANDOM_SEED:33,
    GenParams.REPETITION_PENALTY:1,
    GenParams.MIN_NEW_TOKENS:1,
    GenParams.MAX_NEW_TOKENS:1,
    
}

In [29]:
from ibm_watson_machine_learning.foundation_models import Model

In [30]:
model=Model(

    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id)

In [31]:
model.get_details()

{'model_id': 'google/flan-t5-xxl',
 'label': 'flan-t5-xxl-11b',
 'provider': 'Google',
 'source': 'Hugging Face',
 'functions': [{'id': 'text_generation'}],
 'short_description': 'flan-t5-xxl is an 11 billion parameter model based on the Flan-T5 family.',
 'long_description': 'flan-t5-xxl (11B) is an 11 billion parameter model based on the Flan-T5 family. It is a pretrained T5 - an encoder-decoder model pre-trained on a mixture of supervised / unsupervised tasks converted into a text-to-text format, and fine-tuned on the Fine-tuned Language Net (FLAN) with instructions for better zero-shot and few-shot performance.',
 'terms_url': 'https://huggingface.co/google/flan-t5-xxl/blob/main/README.md',
 'input_tier': 'class_2',
 'output_tier': 'class_2',
 'number_params': '11b',
 'min_shot_size': 0,
 'task_ids': ['question_answering',
  'summarization',
  'retrieval_augmented_generation',
  'classification',
  'generation',
  'extraction'],
 'tasks': [{'id': 'question_answering', 'ratings': {'